In [ ]:
#hide
!pip install -Uqq fastbook
import os
import fastbook
fastbook.setup_book()

In [ ]:
# !pip install fastai --upgrade -q
from fastai.vision.all import *

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!pip install azure-cognitiveservices-search-imagesearch

In [ ]:
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth

def search_images_bing(key, term, min_sz=128, max_images=150):    
     params = {'q':term, 'count':max_images, 'min_height':min_sz, 'min_width':min_sz}
     headers = {"Ocp-Apim-Subscription-Key":key}
     search_url = "https://api.bing.microsoft.com/v7.0/images/search"
     response = requests.get(search_url, headers=headers, params=params)
     response.raise_for_status()
     search_results = response.json()    
     return L(search_results['value'])

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '44e8d9557ef34893b4df6a048b606149')

In [ ]:
plants_types = 'succulent','cactus','tree'
path = Path('plants')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in plants_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} plants')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns=get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
plants = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = plants.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
plants = plants.new(item_tfms=RandomResizedCrop(224, min_scale=0.5) , batch_tfms=aug_transforms(mult=3))
dls = plants.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(13)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:

for idx in cleaner.delete(): 
    print(cleaner.fns[idx])
    cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): 
  shutil.move(str(cleaner.fns[idx]), path/cat)